In [2]:
from langchain_community.llms import Ollama
from langchain.prompts import PromptTemplate
import re
from tqdm.notebook import tqdm
import os
import json

In [3]:
import pandas as pd

df = pd.read_csv('data/train_data_categories.csv')
df = df[df['video_id'] != 'b4d70f82038d1d97f1b3ce2a493d12c8']

def get_first_level(x):
    return x.split(':')[0].strip()

def get_second_level(x):
    if x.count(':') > 0:
        return x.split(':')[1].strip()
    else:
        return '-'
    
def get_third_level(x):
    if x.count(':') == 2:
        return x.split(':')[2].strip()
    else:
        return '-'

df['first_level'] = df['tags'].apply(get_first_level)
df['second_level'] = df['tags'].apply(get_second_level)
df['third_level'] = df['tags'].apply(get_third_level)

In [4]:
prompt_template = PromptTemplate(
    input_variables=["title"],
    template=(
        "Классифицируй к какой категории относится заголовок, верни только подходяющую категорию (может быть 1 или несколько). "
        "Транспорт, Книги и литература, Бизнес и финансы, Карьера, Образование, События и достопримечательности, Семья и отношения, Изобразительное искусство, Еда и напитки, Здоровый образ жизни, Хобби и интересы, Дом и сад, Медицина, Фильмы и анимация, Музыка и аудио, Новости и политика, Личные финансы, Животные, Массовая культура, Недвижимость, Религия и духовность, Наука, Покупки, Спорт, Стиль и красота, Информационные технологии, Телевидение, Путешествия, Игры. "
        "Заголовок видео: \"{title}\". "
    ),
)


llm = Ollama(model="llama3", temperature=0)

In [5]:
def process_title_with_llama(title):
    prompt = prompt_template.format(title=title)
    response = llm.invoke(prompt)
    return response

df['processed_title'] = df['title'].apply(process_title_with_llama)

In [6]:
df

,video_id,title,description,tags,first_level,second_level,third_level,processed_title
0,9007f33c8347924ffa12f922da2a179d,Пацанский клининг. Шоу «ЧистоТачка» | Повелите...,Тяпа и Егор бросили вызов нестареющему «повели...,Массовая культура: Юмор и сатира,Массовая культура,Юмор и сатира,-,"Категории: Телевидение, Массовая культура"
1,9012707c45233bd601dead57bc9e2eca,"СarJitsu. 3 сезон, 6 серия. Нарек Симонян vs Ж...","CarJitsu — бои в формате POP MMA, где вместо р...",События и достопримечательности: Спортивные с...,События и достопримечательности,"Спортивные события, Массовая культура",Юмор и сатира,"Категории: Фильмы и анимация, Массовая культур..."
2,e01d6ebabbc27e323fa1b7c581e9b96a,"Злые языки | Выпуск 1, Сезон 1 | Непорочность ...",Почему Дана Борисова предпочитает молчать о по...,"Массовая культура: Отношения знаменитостей, Ма...",Массовая культура,"Отношения знаменитостей, Массовая культура",Скандалы знаменитостей,"Категории: Фильмы и анимация, Массовая культура"
3,a00b145242be3ebc3b311455e94917af,$1000 шоу | 1 выпуск | Автобоулинг,"В этом выпуске, популярный автоблогер Дима Гор...","Транспорт, Спорт: Автогонки, Массовая культура","Транспорт, Спорт","Автогонки, Массовая культура",-,Транспорт
4,b01a682bf4dfcc09f1e8fac5bc18785a,В РОТ МНЕ НОТЫ #1 ВИТА ЧИКОВАНИ,В первом выпуске «В рот мне ноты» популярная п...,Массовая культура: Юмор и сатира,Массовая культура,Юмор и сатира,-,Музыка и аудио
...,...,...,...,...,...,...,...,...
1044,5fe16aa2869667bc1519e32a4c536b26,"Злые языки | Выпуск 3, Сезон 1 | Эксклюзив Над...",Гость выпуска – Надин Серовски. Ей предстоит о...,Массовая культура:Отношения знаменитостей: Сем...,Массовая культура,Отношения знаменитостей,Семьи знаменитостей,"Категории: Массовая культура, Фильмы и анимация."
1045,4ffa5fbb2a410aa841659d8890ae5e3f,МАКСИМ НАРОДНЫЙ Выпуск №15 ГОТОВИМ «ПОХМЕЛЬНЫЙ...,Предлагаю подписчикам быстро приготовить похме...,Еда и напитки: Кулинария,Еда и напитки,Кулинария,-,"Категории: Кулинария (Еда и напитки), Видео (Ф..."
1046,3fc81df4bfe121ce2bc33dd581f5efeb,Роман Юнусов и блогерка Арина Ростовская пытаю...,В новом выпуске шоу «Спортивный Интерес» Рома ...,Спорт,Спорт,-,-,"Категории: Массовая культура, События и достоп..."
1047,efe0b4139ef82ec270b9e2fe0216214e,Артмеханика. Сезон 2. Выпуск 18. Современные р...,“Артмеханика” представляет своих друзей! Поэты...,"Книги и литература: Поэзия, Музыка и аудио, К...",Книги и литература,"Поэзия, Музыка и аудио, Карьера",-,"Категории: Изобразительное искусство, Музыка и..."
